<a href="https://colab.research.google.com/github/LEESUSUSUSU/Kaggle-practice/blob/main/TItanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow_decision_forests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.14.1
    Uninstalling tensorboard-2.14.1:
      Successfully uninstalled tensorboard-2.14.1
  Attempting uninstall: tensorflow
    Found existing insta

In [3]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
import tensorflow_decision_forests as tfdf

print(f"Found TF-DF {tfdf.__version__}")

Found TF-DF 1.8.1


In [6]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [8]:
train_data = pd.read_csv("/content/drive/MyDrive/캐글/타이타닉/train.csv")
test_data = pd.read_csv("/content/drive/MyDrive/캐글/타이타닉/test.csv")

train_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [9]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


1. 이름을 토큰화합니다. 예를 들어, "Braund, Mr. Owen Harris"는 ["Braund", "Mr.", "Owen", "Harris"]가 됩니다.
2. 티켓에서 접두사를 추출합니다. 예를 들어 티켓 "STON/O2.3101282"는 "STON/O2"가 됩니다.

Part 3: Your first submission
Remember our goal: we want to find patterns in train.csv that help us predict whether the passengers in test.csv survived.

It might initially feel overwhelming to look for patterns, when there's so much data to sort through. So, we'll start simple.

Explore a pattern
Remember that the sample submission file in gender_submission.csv assumes that all female passengers survived (and all male passengers died).

Is this a reasonable first guess? We'll check if this pattern holds true in the data (in train.csv).

Copy the code below into a new code cell. Then, run the cell.


파트 3: 첫 번째 제출
목표를 기억하세요. test.csv의 승객이 생존했는지 예측하는 데 도움이 되는 패턴을 train.csv에서 찾고 싶습니다.

처음에는 정렬할 데이터가 너무 많아서 패턴을 찾는 것이 부담스러울 수 있습니다. 그럼 간단하게 시작해 보겠습니다.

패턴 탐색
gender_submission.csv의 샘플 제출 파일에서는 모든 여성 승객이 생존하고 모든 남성 승객이 사망했다고 가정합니다.

이것이 합리적인 첫 번째 추측입니까? 이 패턴이 데이터(train.csv)에서 유효한지 확인하겠습니다.

아래 코드를 새 코드 셀에 복사하세요. 그런 다음 셀을 실행합니다.

In [11]:
women = train_data.loc[train_data.Sex == 'female']['Survived']
rate_women = sum(women)/len(women)

print("%of women who survived:", rate_women)



%of women who survived: 0.7420382165605095


Before moving on, make sure that your code returns the output above. The code above calculates the percentage of female passengers (in train.csv) who survived.

Then, run the code below in another code cell:

계속 진행하기 전에 코드가 위의 출력을 반환하는지 확인하세요. 위의 코드는 살아남은 여성 승객의 비율(train.csv에서)을 계산합니다.

그런 다음 다른 코드 셀에서 아래 코드를 실행합니다.

In [12]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("%of women who survived:", rate_men)

%of women who survived: 0.18890814558058924


he code above calculates the percentage of male passengers (in train.csv) who survived.

From this you can see that almost 75% of the women on board survived, whereas only 19% of the men lived to tell about it. Since gender seems to be such a strong indicator of survival, the submission file in gender_submission.csv is not a bad first guess!

But at the end of the day, this gender-based submission bases its predictions on only a single column. As you can imagine, by considering multiple columns, we can discover more complex patterns that can potentially yield better-informed predictions. Since it is quite difficult to consider several columns at once (or, it would take a long time to consider all possible patterns in many different columns simultaneously), we'll use machine learning to automate this for us.

위의 코드는 살아남은 남성 승객의 비율(train.csv)을 계산합니다.

이를 통해 탑승한 여성 중 거의 75%가 살아남은 반면 남성 중 19%만이 살아서 그 사실을 알릴 수 있다는 것을 알 수 있습니다. 성별은 생존에 대한 매우 강력한 지표인 것처럼 보이므로,gender_submission.csv의 제출 파일은 나쁜 첫 번째 추측이 아닙니다!

그러나 결국 이 성별 기반 제출은 단 하나의 열에만 예측을 기반으로 합니다. 상상할 수 있듯이 여러 열을 고려하면 잠재적으로 더 나은 정보를 바탕으로 예측할 수 있는 더 복잡한 패턴을 발견할 수 있습니다. 한 번에 여러 열을 고려하는 것은 매우 어렵기 때문에(또는 여러 열의 가능한 모든 패턴을 동시에 고려하는 데 시간이 오래 걸리기 때문에) 기계 학습을 사용하여 이를 자동화하겠습니다.


Your first machine learning model
We'll build what's known as a random forest model. This model is constructed of several "trees" (there are three trees in the picture below, but we'll construct 100!) that will individually consider each passenger's data and vote on whether the individual survived. Then, the random forest model makes a democratic decision: the outcome with the most votes wins!

첫 번째 기계 학습 모델
우리는 랜덤 포레스트(Random Forest) 모델이라고 알려진 것을 구축할 것입니다. 이 모델은 각 승객의 데이터를 개별적으로 고려하고 개인의 생존 여부에 투표하는 여러 개의 "나무"(아래 그림에는 세 개의 나무가 있지만 우리는 100개를 구성하겠습니다!)로 구성됩니다. 그런 다음 랜덤 포레스트 모델은 민주적인 결정을 내립니다. 즉, 가장 많은 표를 얻은 결과가 승리합니다!

he code cell below looks for patterns in four different columns ("Pclass", "Sex", "SibSp", and "Parch") of the data. It constructs the trees in the random forest model based on patterns in the train.csv file, before generating predictions for the passengers in test.csv. The code also saves these new predictions in a CSV file submission.csv.

Copy this code into your notebook, and run it in a new code cell.


아래 코드 셀은 데이터의 네 가지 다른 열("Pclass", "Sex", "SibSp" 및 "Parch")에서 패턴을 찾습니다. test.csv에서 승객에 대한 예측을 생성하기 전에 train.csv 파일의 패턴을 기반으로 랜덤 포레스트 모델에서 트리를 구성합니다. 또한 코드는 이러한 새로운 예측을 CSV 파일 submit.csv에 저장합니다.

이 코드를 노트북에 복사하고 새 코드 셀에서 실행하세요.

In [13]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass","Sex","SibSp","Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X,y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


Make sure that your notebook outputs the same message above (Your submission was successfully saved!) before moving on.

Again, don't worry if this code doesn't make sense to you! For now, we'll focus on how to generate and submit predictions.

Once you're ready, click on the "Save Version" button in the top right corner of your notebook. This will generate a pop-up window.

Ensure that the "Save and Run All" option is selected, and then click on the "Save" button.
This generates a window in the bottom left corner of the notebook. After it has finished running, click on the number to the right of the "Save Version" button. This pulls up a list of versions on the right of the screen. Click on the ellipsis (...) to the right of the most recent version, and select Open in Viewer.
Click on the Data tab on the top of the screen. Then, click on the "Submit" button to submit your results.

계속 진행하기 전에 노트북에서 위와 동일한 메시지(제출물이 성공적으로 저장되었습니다!)를 출력하는지 확인하세요.

다시 한 번 말씀드리지만, 이 코드가 이해가 되지 않더라도 걱정하지 마세요! 지금은 예측을 생성하고 제출하는 방법에 중점을 두겠습니다.

준비가 되면 노트북 오른쪽 상단에 있는 "버전 저장" 버튼을 클릭하세요. 그러면 팝업 창이 생성됩니다.

"모두 저장 및 실행" 옵션이 선택되어 있는지 확인한 다음 "저장" 버튼을 클릭합니다.
그러면 노트북 왼쪽 하단에 창이 생성됩니다. 실행이 완료되면 "버전 저장" 버튼 오른쪽에 있는 숫자를 클릭하세요. 그러면 화면 오른쪽에 버전 목록이 표시됩니다. 최신 버전 오른쪽에 있는 줄임표(...)를 클릭하고 뷰어에서 열기를 선택합니다.
화면 상단의 데이터 탭을 클릭하세요. 그런 다음 "제출" 버튼을 클릭하여 결과를 제출하세요.

In [2]:
!kaggle competitions submit -c titanic -f submission.csv -m "Message"

100% 2.77k/2.77k [00:00<00:00, 3.32kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster